# 12. Intelligent Data Applications

This notebook demonstrates the new **Intelligent Data Applications** framework in elastic-script.

Applications bundle:
- **Sources**: Data indices the application works with
- **Skills**: AI-callable procedures with metadata for MCP/OpenAI function calling
- **Intents**: Natural language patterns mapped to skills
- **Jobs**: Scheduled background tasks
- **Triggers**: Event-driven reactions

## Three Interfaces

1. **Human Interface**: Natural language queries → Intent matching → Skill execution
2. **AI Agent Interface**: MCP/function calling → Skill invocation with typed parameters
3. **Developer Interface**: Direct procedure calls with full control

## 1. Create Supporting Procedures

First, let's create the procedures that our application skills will wrap.

In [ ]:
-- Create a procedure to analyze churn
CREATE PROCEDURE run_churn_analysis(threshold NUMBER)
BEGIN
    DECLARE customers ARRAY;
    DECLARE result ARRAY;
    
    -- Simulated churn analysis (in real app, would query orders-*)
    customers = [
        {'customer_id': 'C001', 'name': 'Alice', 'churn_score': 0.85, 'last_order': '2025-11-01'},
        {'customer_id': 'C002', 'name': 'Bob', 'churn_score': 0.72, 'last_order': '2025-10-15'},
        {'customer_id': 'C003', 'name': 'Carol', 'churn_score': 0.45, 'last_order': '2026-01-10'},
        {'customer_id': 'C004', 'name': 'Dave', 'churn_score': 0.91, 'last_order': '2025-09-20'}
    ];
    
    -- Filter by threshold
    result = ARRAY_FILTER(customers, (c) => c.churn_score >= threshold);
    
    PRINT 'Found ' || ARRAY_LENGTH(result) || ' customers above threshold ' || threshold;
    RETURN result;
END PROCEDURE;

In [ ]:
-- Create a procedure for sales analysis
CREATE PROCEDURE analyze_sales_by_region(region STRING)
BEGIN
    DECLARE sales_data DOCUMENT;
    
    -- Simulated sales data (in real app, would query sales indices)
    sales_data = {
        'region': region,
        'total_revenue': 125000,
        'order_count': 342,
        'avg_order_value': 365.50,
        'top_products': ['Widget Pro', 'Gadget Plus', 'Tool Master'],
        'growth_rate': 0.15
    };
    
    PRINT 'Sales analysis complete for ' || region;
    RETURN sales_data;
END PROCEDURE;

In [ ]:
-- Create a procedure for customer lookup
CREATE PROCEDURE find_customer_by_name(name_pattern STRING)
BEGIN
    DECLARE customers ARRAY;
    
    customers = [
        {'id': 'C001', 'name': 'Alice Johnson', 'email': 'alice@example.com', 'tier': 'gold'},
        {'id': 'C002', 'name': 'Bob Smith', 'email': 'bob@example.com', 'tier': 'silver'},
        {'id': 'C003', 'name': 'Alice Williams', 'email': 'alice.w@example.com', 'tier': 'platinum'}
    ];
    
    -- Filter by name pattern (case-insensitive)
    RETURN ARRAY_FILTER(customers, (c) => INSTR(LOWER(c.name), LOWER(name_pattern)) > 0);
END PROCEDURE;

## 2. Create an Intelligent Data Application

Now let's create an application that bundles these procedures as skills with natural language intents.

In [ ]:
CREATE APPLICATION customer_analytics
DESCRIPTION 'Complete customer analytics platform'
VERSION '1.0.0'
SOURCES (
    orders FROM 'orders-*',
    customers FROM 'customers-*'
)
SKILLS (
    detect_churn(threshold NUMBER) RETURNS ARRAY 
        DESCRIPTION 'Identifies customers likely to churn based on scoring threshold' 
        AS run_churn_analysis(threshold),
    analyze_sales(region STRING) RETURNS DOCUMENT 
        DESCRIPTION 'Analyzes sales patterns and revenue for a geographic region' 
        AS analyze_sales_by_region(region),
    find_customer(name STRING) RETURNS ARRAY 
        DESCRIPTION 'Searches for customers by name' 
        AS find_customer_by_name(name)
)
INTENTS (
    'churn|leaving|at risk|might leave|retention' => detect_churn,
    'sales|revenue|orders|income|earnings' => analyze_sales,
    'find customer|search customer|lookup customer|who is' => find_customer
)
END APPLICATION

## 3. View and Manage Applications

In [ ]:
-- List all applications
SHOW APPLICATIONS

In [ ]:
-- Show application details
SHOW APPLICATION customer_analytics

In [ ]:
-- Show skills available in the application
SHOW APPLICATION customer_analytics SKILLS

In [ ]:
-- Show intents (natural language patterns)
SHOW APPLICATION customer_analytics INTENTS

## 4. Install and Activate the Application

Applications must be installed (set to RUNNING status) before skills and intents are active.

In [ ]:
-- Install the application with optional configuration
INSTALL APPLICATION customer_analytics
CONFIG (
    default_threshold => 0.7,
    default_region => 'North America'
)

In [ ]:
-- Check application status
APPLICATION customer_analytics | STATUS

## 5. Extend an Application

Add new skills and intents dynamically.

In [ ]:
-- First, create a new procedure
CREATE PROCEDURE get_customer_orders(customer_id STRING)
BEGIN
    DECLARE orders ARRAY;
    
    orders = [
        {'order_id': 'O001', 'amount': 150.00, 'date': '2026-01-15', 'status': 'delivered'},
        {'order_id': 'O002', 'amount': 275.50, 'date': '2026-01-18', 'status': 'shipped'}
    ];
    
    RETURN orders;
END PROCEDURE;

In [ ]:
-- Add a new skill to the application
EXTEND APPLICATION customer_analytics
ADD SKILL get_orders(customer_id STRING) RETURNS ARRAY 
    DESCRIPTION 'Retrieves order history for a customer' 
    AS get_customer_orders(customer_id)

In [ ]:
-- Add a new intent mapping
EXTEND APPLICATION customer_analytics
ADD INTENT 'orders|purchases|bought|order history' => get_orders

## 6. Application Lifecycle Management

In [ ]:
-- Pause the application (skills and intents become inactive)
APPLICATION customer_analytics | PAUSE

In [ ]:
-- Check status after pausing
APPLICATION customer_analytics | STATUS

In [ ]:
-- Resume the application
APPLICATION customer_analytics | RESUME

In [ ]:
-- Alternative: Enable/Disable via ALTER
-- ALTER APPLICATION customer_analytics DISABLE
-- ALTER APPLICATION customer_analytics ENABLE

## 7. REST API Access

The Skills and Intent APIs provide programmatic access for AI agents.

### Skills API (for MCP/OpenAI Function Calling)

```bash
# List all skills (MCP tool format)
curl -u elastic-admin:elastic-password http://localhost:9200/_escript/skills

# Get skill details
curl -u elastic-admin:elastic-password http://localhost:9200/_escript/skills/detect_churn

# Invoke a skill
curl -u elastic-admin:elastic-password http://localhost:9200/_escript/skills/detect_churn/_invoke \
  -H 'Content-Type: application/json' \
  -d '{"threshold": 0.8}'
```

### Intent API (for Natural Language)

```bash
# Match intent without executing
curl -u elastic-admin:elastic-password http://localhost:9200/_escript/intent/_match \
  -H 'Content-Type: application/json' \
  -d '{"query": "Show me customers who might be leaving"}'

# Match and execute intent
curl -u elastic-admin:elastic-password http://localhost:9200/_escript/intent \
  -H 'Content-Type: application/json' \
  -d '{"query": "Show me customers who might be leaving", "params": {"threshold": 0.7}}'
```

## 8. Cleanup

In [ ]:
-- Drop the application
DROP APPLICATION customer_analytics

In [ ]:
-- Drop the procedures
DELETE PROCEDURE run_churn_analysis;
DELETE PROCEDURE analyze_sales_by_region;
DELETE PROCEDURE find_customer_by_name;
DELETE PROCEDURE get_customer_orders;

## Summary

The Intelligent Data Applications framework provides:

1. **CREATE APPLICATION** - Bundle procedures, skills, intents, sources, jobs, triggers
2. **INSTALL APPLICATION** - Activate with configuration
3. **EXTEND APPLICATION** - Add skills/intents dynamically
4. **Skills REST API** - `/_escript/skills` for MCP/function calling
5. **Intent REST API** - `/_escript/intent` for natural language queries
6. **Lifecycle management** - PAUSE/RESUME/ENABLE/DISABLE

This enables elastic-script to serve as a unified platform for:
- Human users (natural language)
- AI agents (typed function calls)
- Developers (direct procedure invocation)